### covid-19 JH data ingestion<br>
author   : santosh<br>
date     : 05.01.2020<br>
purpose  : ingest covid-19 data, reformat and generate analysis dataset<br>
instance : local<br>

<font color='tan'> setup

In [1]:
#import dependencies
#data wrangling
import numpy as np
import pandas as pd
import dfply as dfp
import datetime as dt
#plot
# import matplotlib.pyplot as plt
# plt.style.use('dark_background')
# import seaborn as sb
#helper funcs
from helper_funcs import isUnique

In [2]:
#display options
pd.options.display.html.table_schema = True
pd.set_option('display.max_rows', 999)
# %matplotlib inline

<font color='tan'> ingest data

In [3]:
#func to read csv file
def ingestJH(ipfilename, colname):
    """
    this func ingests JH covid-19 data and returns a scrubbed dataframe
    """
    #read csv and drop 1st row
    df = pd.read_csv(ipfilename, sep=',',skiprows=0, parse_dates=['Date'])
    df.drop(index=0, axis=0, inplace=True)
    
    #scrub fields
    df.rename(columns={'Country/Region':'Country_Region',
                       'Province/State':'Province_State',
                       'Value':colname,
                       'ISO 3166-1 Alpha 3-Codes':'ISO3166-1Alpha3-Codes',
                       'Region Code':'Region_Code',
                       'Sub-region Code':'Sub_Region_Code',
                       'Intermediate Region Code':'Intermediate_Region_Code'}, inplace=True)
    
    #recast data types
    df['Date']  = pd.to_datetime(df['Date'], errors='coerce')
    df['Lat']   = pd.to_numeric(df['Lat'],   errors='coerce')
    df['Long']  = pd.to_numeric(df['Long'],  errors='coerce')
    df[colname] = pd.to_numeric(df[colname], errors='coerce')
    
    #make country value uppercase
    df['Country_Region'] = df['Country_Region'].apply(lambda x:x.upper())
    
    #aggregate across country
    df = df.groupby(['Country_Region','Date']).agg({colname:'sum'}).reset_index()
    print('\nisData Unique in Country and Date: \n')
    print(isUnique(df, ['Country_Region','Date']))
    
    #return df
    return df

In [4]:
df_confirmed = ingestJH('../data/time_series_covid19_confirmed_global_narrow.csv','Confirmed')
df_deaths = ingestJH('../data/time_series_covid19_deaths_global_narrow.csv','Deaths')
df_recovered = ingestJH('../data/time_series_covid19_recovered_global_narrow.csv','Recovered')


isData Unique in Country and Date: 

True

isData Unique in Country and Date: 

True

isData Unique in Country and Date: 

True


In [5]:
#merge confirmed, deaths and recovered data into a single dataframe
df_all = pd.merge(df_confirmed,
                  df_deaths,
                  on=['Country_Region','Date'],
                  how='left',
                  indicator=False)

df_all = pd.merge(df_all,
                  df_recovered,
                  on=['Country_Region','Date'],
                  how='left',indicator=False)

In [6]:
#print sample recs
df_all.loc[df_all['Country_Region'] == 'US'].tail()

,Country_Region,Date,Confirmed,Deaths,Recovered
18432,US,2020-04-29,1039909,60967,120720
18433,US,2020-04-30,1069424,62996,153947
18434,US,2020-05-01,1103461,64943,164015
18435,US,2020-05-02,1132539,66369,175382
18436,US,2020-05-03,1158040,67682,180152


In [7]:
#get list of all countries in dataframe
list_country = df_all['Country_Region'].unique().tolist()
print('\nnumber of unique countries in list: ', len(list_country))


number of unique countries in list:  187


<font color='tan'> append lagged values

In [8]:
#func to append lag of infusion counts
def getLagValues(country):
    """
    this func appends lagged cumulative infusion counts
    for an input list of countries supplied
    """
    #filter for required country
    stack = df_all.loc[df_all['Country_Region'] == country]\
    [['Country_Region','Date','Confirmed','Deaths','Recovered']]\
    .sort_values(by=['Date'], ascending=True)
    
    #check for uniqueness
    bool_check = isUnique(stack, ['Date'])
    if bool_check == False:
        print('\naborted due to uniqueness check fail: ', country)
        stop
    else:
        pass
    
    #append lag of cum_inf_count
    stack['Confirmed_Lag'] = stack['Confirmed'].shift(1)
    stack['Deaths_Lag']    = stack['Deaths'].shift(1)
    stack['Recovered_Lag'] = stack['Recovered'].shift(1)
    
    #compute new cases
    stack['Confirmed_New'] = (stack['Confirmed'] - stack['Confirmed_Lag'])
    stack['Confirmed_New'].fillna(0, inplace=True)
    stack['Deaths_New'] = (stack['Deaths'] - stack['Deaths_Lag'])
    stack['Deaths_New'].fillna(0, inplace=True)
    stack['Recovered_New'] = (stack['Recovered'] - stack['Recovered_Lag'])
    stack['Recovered_New'].fillna(0, inplace=True)
    #drop unwanted cols
    stack.drop(columns=['Confirmed_Lag','Deaths_Lag','Recovered_Lag'], inplace=True)
    
    #return data
    return stack

In [9]:
#run lag func for all countries
df_final = pd.DataFrame() #empty dataframe

#iterate over country list
for item in list_country:
    temp = getLagValues(item)
    df_final = pd.concat([df_final, temp], axis=0, sort=True)

In [10]:
df_final.head()

,Confirmed,Confirmed_New,Country_Region,Date,Deaths,Deaths_New,Recovered,Recovered_New
0,0,0.0,AFGHANISTAN,2020-01-22,0,0.0,0,0.0
1,0,0.0,AFGHANISTAN,2020-01-23,0,0.0,0,0.0
2,0,0.0,AFGHANISTAN,2020-01-24,0,0.0,0,0.0
3,0,0.0,AFGHANISTAN,2020-01-25,0,0.0,0,0.0
4,0,0.0,AFGHANISTAN,2020-01-26,0,0.0,0,0.0


<font color='tan'> append year, month, and year_month

In [11]:
df_final['year']  = df_final['Date'].dt.year
df_final['month'] = df_final['Date'].dt.month
df_final['year_month'] = df_final['year'].astype(str) +'_'+ df_final['month'].astype(str)
df_final.head()

,Confirmed,Confirmed_New,Country_Region,Date,Deaths,Deaths_New,Recovered,Recovered_New,year,month,year_month
0,0,0.0,AFGHANISTAN,2020-01-22,0,0.0,0,0.0,2020,1,2020_1
1,0,0.0,AFGHANISTAN,2020-01-23,0,0.0,0,0.0,2020,1,2020_1
2,0,0.0,AFGHANISTAN,2020-01-24,0,0.0,0,0.0,2020,1,2020_1
3,0,0.0,AFGHANISTAN,2020-01-25,0,0.0,0,0.0,2020,1,2020_1
4,0,0.0,AFGHANISTAN,2020-01-26,0,0.0,0,0.0,2020,1,2020_1


<font color='tan'> output as CSV file

In [12]:
opfile = '../data/covid19_country_analysis_data.csv'
df_final.to_csv(opfile, index=False)

<font color='tan'> end